In [1]:
import pandas as pd
import glob
import numpy as np
import re
# Import the tabulate library
from tabulate import tabulate
from uszipcode import SearchEngine
import concurrent.futures

In [2]:
# Load dataset
file_paths = glob.glob('C:/Users/SANTIAGO/OneDrive - Nova Scotia Community College/4 Winter Semester 2024/DBAS 3018 Data Movement and Integration/Assignment 2/Datasets/Revised Dataset/Bike Donations/*.csv')

# Read each csv file
dataframes = [pd.read_csv(file, low_memory=False) for file in file_paths]

# Combine csv files into one
donations_df = pd.concat(dataframes, ignore_index=True)

In [3]:
shape = donations_df.shape[0]
count = '{:,}'.format(shape)
count

'3,807,376'

In [4]:
donations_df.head()

,Security Category Name,Event ID,Public Event Name,Fiscal Year,Event Date,Campaign Title,Campaign ID,Gift Amount($),Date Recorded,Gift Type,...,Team Creation Date,Original Value Transacted,Net Transaction Amount,Ledger Transaction Amount,Proxy Type,User Confirmation Code,Source Code Type,Source Code Text,Sub Source Code Text,Batch ID
0,ALC Bike Events,17966,2012 Bike MS: Bike to the Battlefield,2013,10/6/2012 11:22,ALC 2012 Bike MS North Alabama,32881,100.0,9/14/2012 17:34,online,...,9/2/2012 18:40,100.0,100,100,TeamRaiser Participant,1638-32881-1-10783905-11126873,Web Link,boundlessfundraising,bfstream,NaN
1,ALC Bike Events,17966,2012 Bike MS: Bike to the Battlefield,2013,10/6/2012 11:22,ALC 2012 Bike MS North Alabama,32881,50.0,9/21/2012 14:10,online,...,9/2/2012 18:40,50.0,50,50,TeamRaiser Participant,1638-32881-1-10829724-11174780,Web Link,boundlessfundraising,bfstream,NaN
2,ALC Bike Events,17966,2012 Bike MS: Bike to the Battlefield,2013,10/6/2012 11:22,ALC 2012 Bike MS North Alabama,32881,25.0,9/21/2012 10:43,online,...,9/2/2012 18:40,25.0,25,25,TeamRaiser Participant,1638-32881-1-10827256-11172221,Web Link,boundlessfundraising,bfstream,NaN
3,ALC Bike Events,17966,2012 Bike MS: Bike to the Battlefield,2013,10/6/2012 11:22,ALC 2012 Bike Clinton,32494,70.0,10/2/2012 15:58,online,...,2/29/2012 15:57,70.0,70,70,TeamRaiser Participant,1638-32494-1-10893149-11241474,6,36ohk6dgmcd1n-c.c.yom.mail.yahoo.net,/om/api/1.0/openmail.app.invoke/36ohk6dgmcd1n/...,NaN
4,ALC Bike Events,17966,2012 Bike MS: Bike to the Battlefield,2013,10/6/2012 11:22,ALC 2012 Bike MS North Alabama,32881,25.0,8/17/2012 14:15,online,...,2/29/2012 15:57,25.0,25,25,TeamRaiser Participant,1638-32881-1-10610796-10946043,Web Link,boundlessfundraising,bfstream,NaN


In [5]:
dictionary = []
# Iterate through columns

# total rows
total_rows = len(donations_df)

for col in donations_df.columns:
    data_type = donations_df[col].dtype
    length = None
    if data_type == object:
        length = donations_df[col].str.len().max()
    
    length_format = '{:,.0f}'.format(length) if length is not None else None
    
    num_nulls = donations_df[col].isnull().sum()
    num_nulls_format = '{:,}'.format(num_nulls)
    percentage_null = (num_nulls/total_rows) * 100
    percentage_null_format = '{:,.2f}%'.format(percentage_null)
    num_distinct = donations_df[col].nunique()
    num_distinct_format = '{:,}'.format(num_distinct)
    
    # Initialize min and max values
    min_value = None
    max_value = None
    
    # Check if column is numeric
    if np.issubdtype(data_type, np.number):
        min_value = donations_df[col].min()
        max_value = donations_df[col].max()
        
    # Calculate avergae for numeric columns
    average = donations_df[col].mean() if np.issubdtype(data_type, np.number) else None
        
    #Sample values
    sample_values = donations_df[col].dropna().sample(5).tolist()
    
    #Append all info to dictionary list
    dictionary.append([col, data_type, length_format, num_nulls_format, percentage_null_format, num_distinct_format, min_value, max_value, average, sample_values])
    
# Create a df for the dictionary
dictionary_df = pd.DataFrame(dictionary, columns=['Column', 'Data_Type', 'Max_Character_Length', 'Null_Value_Count', 'Null_Percentage','Distinct_Value_Count', 'Min_Value', 'Max_Value','Average','Sample_Values'])

# So that Sample_Values are not truncated
pd.set_option('display.max_colwidth', 300)

# sort = dictionary_df.sort_values(by='Null_Value_Count')
# sort

dictionary_df

,Column,Data_Type,Max_Character_Length,Null_Value_Count,Null_Percentage,Distinct_Value_Count,Min_Value,Max_Value,Average,Sample_Values
0,Security Category Name,object,25,0,0.00%,162,NaN,NaN,NaN,"[MNM Bike Events, MIG Bike Events, COC Bike Events, MNM Bike Events, NYN Bike Events]"
1,Event ID,object,38,0,0.00%,508,NaN,NaN,NaN,"[28263, 22278, 18743, 27003, 25223]"
2,Public Event Name,object,104,0,0.00%,510,NaN,NaN,NaN,"[Bike MS: Willamette Valley 2016, Bike MS: Willamette Valley 2015, Bike MS: Colorado 2017 powered locally by Anthem, New Jersey, Cherry Hill Bike MS: City to Shore 2014, Bike MS: Cape Cod Getaway 2017]"
3,Fiscal Year,object,19,0,0.00%,9,NaN,NaN,NaN,"[2016, 2013, 2016, 2013, 2016]"
4,Event Date,object,26,42,0.00%,494,NaN,NaN,NaN,"[4/20/2013 12:05, 4/16/2016 13:18, 6/29/2013 8:42, 4/12/2014 16:41, 9/12/2015 13:32]"
5,Campaign Title,object,70,0,0.00%,296,NaN,NaN,NaN,"[NYN 2012 Bike MS New York City, ORC Bike Donation Campaign, CTN Bike Donation Campaign, Bike MS: Breakaway to the Beach 2014, Texas, Houston - 2014 BP MS 150]"
6,Campaign ID,object,9,0,0.00%,328,NaN,NaN,NaN,"[44959, 35592, 35352, 38872, 35352]"
7,Gift Amount($),object,17,0,0.00%,"17,516",NaN,NaN,NaN,"[35, 100, 50, 15, 50.0]"
8,Date Recorded,object,23,0,0.00%,"1,382,882",NaN,NaN,NaN,"[2/13/2013 21:24, 9/10/2016 16:34, 8/15/2015 8:20, 5/26/2015 12:33, 7/7/2014 20:14]"
9,Gift Type,object,11,0,0.00%,8,NaN,NaN,NaN,"[offline, online, online, online, offline]"


In [6]:
donations_df.columns

Index(['Security Category Name', 'Event ID', 'Public Event Name',
       'Fiscal Year', 'Event Date', 'Campaign Title', 'Campaign ID',
       'Gift Amount($)', 'Date Recorded', 'Gift Type', 'Gift Payment Method',
       'Offline Status', 'Soft Credit Type', 'Is Registration?',
       'Donor ConsID', 'Donor Member ID', 'Donor Affiliate Code',
       'Donor Gender', 'Donor Accept Email', 'Donor Opt-out Date',
       'Donor Opt-out Method', 'Donor Email Status', 'Donor City',
       'Donor State', 'Donor County', 'Donor ZIP', 'Donor Employer',
       'Donor Connection to MS', 'Participant Contact ID',
       'Participant Member ID', 'Participation Type Name',
       'Registration Active Status', 'Registration Date',
       'Participant Goal($)', 'Is Prior Participant', 'Is Team Captain',
       'Additional Gift Amount($)', 'Team Name', 'Team', 'Team Creation Date',
       'Original Value Transacted', 'Net Transaction Amount',
       'Ledger Transaction Amount', 'Proxy Type', 'User Confirm

# Fiscal Year

In [7]:
# Fiscal Year dirty value
max_length_value = donations_df['Fiscal Year'].astype(str).apply(len).idxmax()
max_length = donations_df['Fiscal Year'].astype(str).apply(len).max()
value_with_max_length = donations_df.at[max_length_value, 'Fiscal Year']
value_with_max_length

# List all values 
long_year_values = donations_df[donations_df['Fiscal Year'].astype(str).apply(len) > 4]['Fiscal Year'].unique()

long_year_values



array(['9/24/2016 9:27', '4/30/2016 12:49', '2017(7/15/2017 6:29'],
      dtype=object)

In [8]:
# Function to extract year
def extract_year(value):
    match = re.search(r'\b\d{4}\b', str(value))
    return match.group(0) if match else None

# Apply the function to the 'Fiscal Year' column
donations_df['Fiscal Year'] = donations_df['Fiscal Year'].apply(extract_year)

# Check the cleaned data
max_char = max(len(str(year)) for year in donations_df['Fiscal Year'])
max_char

4

In [9]:
# Check for special characters in the 'Fiscal Year' column
special = donations_df['Fiscal Year'].astype(str).str.contains(r'[^\w\s]', regex=True)

rows_special = donations_df[special]
rows_special

,Security Category Name,Event ID,Public Event Name,Fiscal Year,Event Date,Campaign Title,Campaign ID,Gift Amount($),Date Recorded,Gift Type,...,Team Creation Date,Original Value Transacted,Net Transaction Amount,Ledger Transaction Amount,Proxy Type,User Confirmation Code,Source Code Type,Source Code Text,Sub Source Code Text,Batch ID


In [10]:
# Change dtype to int
donations_df['Fiscal Year'] = pd.to_numeric(donations_df['Fiscal Year'], errors='coerce').astype('Int64')

In [11]:
donations_df['Fiscal Year']

0          2013
1          2013
2          2013
3          2013
4          2013
           ... 
3807371    2017
3807372    2017
3807373    2017
3807374    2017
3807375    2017
Name: Fiscal Year, Length: 3807376, dtype: Int64

# Date Recorded	

In [12]:
# Date Recorded dirty value
max_length_value = donations_df['Date Recorded'].astype(str).apply(len).idxmax()
max_length = donations_df['Date Recorded'].astype(str).apply(len).max()
value_with_max_length = donations_df.at[max_length_value, 'Date Recorded']
value_with_max_length

# List all values 
long_year_values = donations_df[donations_df['Date Recorded'].astype(str).apply(len) > 16]['Date Recorded'].unique()

long_year_values

array(['10/4/2012 12:26(offline', '9/5/2013 13:28(offline',
       '10/12/2016 7:07(offline'], dtype=object)

In [13]:
# Check max no. of char should be 15 to 18
max_char = max(len(str(year)) for year in donations_df['Date Recorded'])
max_char

23

In [14]:
donations_df['Date Recorded'] = donations_df['Date Recorded'].astype(str)

In [15]:
# Correct the 'Date Recorded' column by removing '(offline'
donations_df['Date Recorded'] = donations_df['Date Recorded'].str.replace(r'\(offline.*', '', regex=True)

In [16]:
donations_df['Date Recorded']

0          9/14/2012 17:34
1          9/21/2012 14:10
2          9/21/2012 10:43
3          10/2/2012 15:58
4          8/17/2012 14:15
                ...       
3807371     5/16/2017 8:57
3807372    5/18/2017 16:49
3807373    5/17/2017 20:07
3807374      5/2/2017 7:57
3807375      6/7/2017 8:49
Name: Date Recorded, Length: 3807376, dtype: object

In [17]:
# convert to datetime
# 'errors='coerce'' will handle invalid date formats by converting them to NaT
donations_df['Date Recorded'] = pd.to_datetime(donations_df['Date Recorded'].str.split('(').str[0], errors='coerce')

In [18]:
# Display the DataFrame with the cleaned up dates
donations_df['Date Recorded']

0         2012-09-14 17:34:00
1         2012-09-21 14:10:00
2         2012-09-21 10:43:00
3         2012-10-02 15:58:00
4         2012-08-17 14:15:00
                  ...        
3807371   2017-05-16 08:57:00
3807372   2017-05-18 16:49:00
3807373   2017-05-17 20:07:00
3807374   2017-05-02 07:57:00
3807375   2017-06-07 08:49:00
Name: Date Recorded, Length: 3807376, dtype: datetime64[ns]

In [19]:
# List all NaT values
nat_values = donations_df[donations_df['Date Recorded'].isna()]

# Count the NaT values
nat_count = nat_values.shape[0]

# Print only 'Date Recorded' with NaT values
print("Date Recorded with NaT values:")
print(nat_values['Date Recorded'])

Date Recorded with NaT values:
292537    NaT
1065707   NaT
2133961   NaT
2161980   NaT
2951655   NaT
2975192   NaT
3109122   NaT
3142276   NaT
3376382   NaT
Name: Date Recorded, dtype: datetime64[ns]


In [20]:
# Replace NaT values with NULL
donations_df['Date Recorded'].fillna(value=np.nan, inplace=True)

# Gift Type

In [21]:
max_length_value = donations_df['Gift Type'].astype(str).apply(len).idxmax()
max_length = donations_df['Gift Type'].astype(str).apply(len).max()
value_with_max_length = donations_df.at[max_length_value, 'Gift Type']
value_with_max_length

unique_values = donations_df['Gift Type'].unique()
print(unique_values)

['online' 'offline' 'ofbline' 'Check' 'Credit Card' 'ojline' 'onlina'
 'onhine']


In [22]:
# Gift Type Cleanup

# Define a mapping of incorrect values to their corrected forms
corrections = {
    'ofbline': 'Offline',
    'ojline': 'Online',
    'onlina': 'Online',
    'onhine': 'Online',
    'offline':'Offline',
    'online':'Online'
}

# Apply the corrections to the 'Gift Type' column
donations_df['Gift Type'] = donations_df['Gift Type'].replace(corrections)

In [23]:
# Check the cleaned data
max_char = max(len(str(type)) for type in donations_df['Gift Type'])
max_char

11

In [24]:
donations_df['Gift Type']

0          Online
1          Online
2          Online
3          Online
4          Online
            ...  
3807371    Online
3807372    Online
3807373    Online
3807374    Online
3807375    Online
Name: Gift Type, Length: 3807376, dtype: object

# Team (TeamID)

In [25]:
# Team (ID) dirty value
max_length_value = donations_df['Team'].astype(str).apply(len).idxmax()
max_length = donations_df['Team'].astype(str).apply(len).max()
value_with_max_length = donations_df.at[max_length_value, 'Team']
value_with_max_length

# filtered values 
values = donations_df[donations_df['Team'].astype(str).apply(len) > 6][['Team','Team Name']]
values

,Team,Team Name
46998,6/14/2012 13:42,304556
175502,12/12/2012 8:55,315347
321402,304170(6/7/2012 12:03,Real Estate Riders
485629,12/30/2012 21:57,316590
652437,10/12/2012 12:03,311173
992076,10/9/2013 9:13,356595
1065707,10/11/2013 15:09,356809
1336119,9/2/2013 19:46,352925
1551042,12/29/2013 22:32,362917
1774627,11/18/2014 17:34,407723


In [26]:
# Filter the DataFrame based on the conditions for 'Team' and 'Team Name'
# Filter conditions
condition = (donations_df['Team'].astype(str).apply(len) > 6) & \
            (donations_df['Team Name'].astype(str).apply(len) <= 6)

# Display the filtered values
condition

# Update the 'Team' column only for the filtered rows
donations_df.loc[condition, 'Team'] = donations_df.loc[condition, 'Team Name']

print(donations_df.loc[condition, ['Team', 'Team Name']])

           Team Team Name
46998    304556    304556
175502   315347    315347
485629   316590    316590
652437   311173    311173
992076   356595    356595
1065707  356809    356809
1336119  352925    352925
1551042  362917    362917
1774627  407723    407723
1842656  404915    404915
1943101  404363    404363
2108177  448527    448527
2159702  406486    406486
2224841  412791    412791
2487511  400439    400439
2492079  466902    466902
2721961  450755    450755
2833755  441031    441031
2868951  460427    460427
2886801  452208    452208
2938962  451807    451807
2940588  451996    451996
2951655  452046    452046
2975192  452939    452939
3023071  452850    452850
3055540  452316    452316
3115077  452621    452621
3142276  472164    472164
3183686  505827    505827
3222094  477018    477018
3267514  493587    493587
3376382  500893    500893
3396540  490783    490783
3459446  504113    504113
3766007  517985    517985


In [27]:
# Define the condition for filtering
condition = donations_df['Team'].astype(str).apply(len) > 6

# Apply the string manipulation directly to the 'Team' column of the original DataFrame
donations_df.loc[condition, 'Team'] = donations_df.loc[condition, 'Team'].str.split('(').str[0]

# Now, if you want to see the updated rows:
print(donations_df[condition][['Team', 'Team Name']])


           Team           Team Name
321402   304170  Real Estate Riders
3189088  493858  Power of the Pedal


In [28]:
# Change dtype to int
donations_df['Team'] = pd.to_numeric(donations_df['Team'], errors='coerce').astype('Int64')

In [29]:
num_nulls = donations_df['Team'].isnull().sum()
num_nulls

505044

In [30]:
donations_df['Team']

0          307893
1          307893
2          307893
3          286443
4          286443
            ...  
3807371    525976
3807372    525976
3807373    525976
3807374    529051
3807375    531094
Name: Team, Length: 3807376, dtype: Int64

# Donor Gender

In [31]:
# Donor Gender dirty value
max_length_value = donations_df['Donor Gender'].astype(str).apply(len).idxmax()
max_length = donations_df['Donor Gender'].astype(str).apply(len).max()
value_with_max_length = donations_df.at[max_length_value, 'Donor Gender']
value_with_max_length

unique_values = donations_df['Donor Gender'].unique()
unique_values

array(['Female', 'Male', nan, 'FALSE', 'TRUE', 'U', 'Iale', 'F', 'Feiale',
       'M', 'Femala', 'Famale'], dtype=object)

In [32]:
# Create a mapping of values to be replaced
replacement_map = {
    'F': 'Female', 
    'U': 'Unknown', 
    'M': 'Male', 
    'Femala': 'Female', 
    'Famale': 'Female', 
    'Feiale': 'Female', 
    'Iale': 'Male',
    'FALSE': 'Unknown',
    'TRUE': 'Unknown'
}

# Replace the values in the 'Donor Gender' column
donations_df['Donor Gender'] = donations_df['Donor Gender'].replace(replacement_map)

# Handle NaN values
donations_df['Donor Gender'] = donations_df['Donor Gender'].fillna('Unknown')

# Check the unique values after replacement
unique_values_after = donations_df['Donor Gender'].unique()
print(unique_values_after)

['Female' 'Male' 'Unknown']


In [33]:
# Check the cleaned data
max_char = max(len(str(gender)) for gender in donations_df['Donor Gender'])
max_char

7

In [34]:
# Display the 'Donor Gender' and 'Donor ConsID' columns for the filtered DataFrame
x = (donations_df[['Donor Gender', 'Donor ConsID','Fiscal Year']])
x

,Donor Gender,Donor ConsID,Fiscal Year
0,Female,7828995,2013
1,Female,11527263,2013
2,Female,11527263,2013
3,Male,10244498,2013
4,Female,11415664,2013
...,...,...,...
3807371,Female,15614806,2017
3807372,Female,16085762,2017
3807373,Female,16083016,2017
3807374,Unknown,16033523,2017


# Donor ZIP

In [35]:
# Filter for rows where 'Donor ZIP' is null and both 'Donor City' and 'Donor State' are not null
filtered_rows = donations_df[(donations_df['Donor ZIP'].isnull()) & 
                     (donations_df['Donor City'].notnull()) & 
                     (donations_df['Donor State'].notnull())]

# Selecting only 'Donor City' and 'Donor State' columns from the filtered rows
result = filtered_rows[['Donor City', 'Donor State','Donor ZIP']]
result

,Donor City,Donor State,Donor ZIP
46998,CA,San Diego,NaN
209938,MO,Greene,NaN
1065707,KS,Johnson,NaN
1419278,Askeaton,CA,NaN
1499861,Dubai,YT,NaN
1724539,Margaye,IA,NaN
1774627,Denver(CO,Denver,NaN
1916408,Boston,MA,NaN
2043479,Bloomfield,MI,NaN
2487511,WI,Sauk,NaN


In [36]:
# Initialize the SearchEngine
search = SearchEngine()

# Define a function to check the validity of ZIP codes
def check_validity(zip_codes):
    invalid_zip_list = []
    for zip_code in zip_codes:
        # Check if the ZIP code is not missing and is valid (you can replace this with your validation logic)
        if not pd.isna(zip_code) and (len(zip_code) != 5 or not zip_code.isdigit()):
            invalid_zip_list.append(zip_code)
    return invalid_zip_list

# Split the ZIP codes into chunks
num_workers = 4  # Adjust the number of worker threads as needed
chunks = np.array_split(donations_df['Donor ZIP'].to_numpy().transpose(), num_workers)

# Use concurrent.futures to process chunks in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
    results = list(executor.map(check_validity, chunks))

# Combine the results into a single list of invalid ZIP codes
invalid_zips = []
for result in results:
    invalid_zips.extend(result)

# Create a DataFrame containing the invalid ZIP codes
invalid_zips_df = pd.DataFrame({'Invalid ZIP Codes': invalid_zips})

# Filter the original DataFrame to only include rows with invalid ZIP codes
invalid_zips_data = donations_df[donations_df['Donor ZIP'].isin(invalid_zips)]

# Create a new DataFrame with the selected columns and make a copy
invalid_zips_df = invalid_zips_data[['Donor City', 'Donor County', 'Donor State', 'Donor ZIP']].copy()

# Rename the 'Donor ZIP' column to 'Invalid ZIP Codes'
invalid_zips_df.rename(columns={'Donor ZIP': 'Invalid ZIP Codes'}, inplace=True)

# Display the DataFrame
invalid_zips_df

,Donor City,Donor County,Donor State,Invalid ZIP Codes
161,Barrington,Bristol,RI,2806
189,Calgary,NaN,AB,T2S0S7
326,Fairfield,Fairfield,CT,6828
641,Guildford,NaN,NaN,GU24RG
659,London,NaN,NaN,W120JT
...,...,...,...,...
3806896,Russell,NaN,MB,R0J 1W0
3807233,Chatham,Morris,NJ,7928
3807247,Toronto,NaN,ON,M6G 2A8
3807273,Long Valley,Morris,NJ,7853


In [37]:
num_nulls = donations_df['Donor ZIP'].isnull().sum()
num_nulls

1007

In [38]:
donations_df['Donor ZIP']

0          39202
1          35209
2          35209
3          75034
4          78737
           ...  
3807371    53711
3807372    52302
3807373    53719
3807374    55406
3807375    53186
Name: Donor ZIP, Length: 3807376, dtype: object

# Donor County

In [39]:
# Using regular expressions to filter for values that contain a number in 'Donor County'
values_with_numbers = donations_df[donations_df['Donor County'].str.contains(r'\d', na=False)]
# Set display option to show all columns
pd.set_option('display.max_columns', None)
values_with_numbers.head(2)

,Security Category Name,Event ID,Public Event Name,Fiscal Year,Event Date,Campaign Title,Campaign ID,Gift Amount($),Date Recorded,Gift Type,Gift Payment Method,Offline Status,Soft Credit Type,Is Registration?,Donor ConsID,Donor Member ID,Donor Affiliate Code,Donor Gender,Donor Accept Email,Donor Opt-out Date,Donor Opt-out Method,Donor Email Status,Donor City,Donor State,Donor County,Donor ZIP,Donor Employer,Donor Connection to MS,Participant Contact ID,Participant Member ID,Participation Type Name,Registration Active Status,Registration Date,Participant Goal($),Is Prior Participant,Is Team Captain,Additional Gift Amount($),Team Name,Team,Team Creation Date,Original Value Transacted,Net Transaction Amount,Ledger Transaction Amount,Proxy Type,User Confirmation Code,Source Code Type,Source Code Text,Sub Source Code Text,Batch ID
46998,CAS Bike Events,18684,Bike MS Bay to Bay Tour 2012,2013,10/20/2012 15:18,CAS 2012 Bike MS,33413,100.0,2012-10-04 12:26:00,Check,confirmed,Teamraiser Participant Gift,FALSE,11567220,43323309,CAS,Male,Unknown,4/28/2015 17:17,Unsubscribe Link,Good,San Diego,CA,San Diego,92130,NaN,Sibling has MS,14818518,58403331.0,Participant - No Bus,Active,6/14/2012 13:42,2000,NaN,TRUE,0,DAs Against MS,304556,304556,100,100.0,100,TeamRaiser Participant,1638-33413-1-10905487-11254336,NaN,NaN,NaN,CAS09272012RBDB893,NaN
175371,ILD Bike Events,20718,ILD Bike MS: Tour de Farms 2013,2013,6/22/2013 11:50,ILD Bike MS 2013 Donation Campaign,36072,255.0,2013-08-22 13:34:00,Offline,Cash,confirmed,Teamraiser Team Gift,FALSE,5583785,45956215,ILD,Female,FALSE,6/10/2013 19:32,Data Sync,Unknown,Chicago,IL,Cook - District 11,60643,LLS,I have MS,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,Team Kroeschell,314577,12/3/2012 8:09,255.0,255,255,TeamRaiser Team,1638-36072-1-12425069-12875974,6,intranet.nmss.org,/Pages/Home.aspx,ILD082113B114JF


In [40]:
num_nulls = donations_df['Donor County'].isnull().sum()
num_nulls

74047

In [41]:
# Initialize the SearchEngine
search = SearchEngine()

# Create a function to get the county based on ZIP code
def get_county(zipcode):
    result = search.by_zipcode(zipcode)
    if result:
        return result.to_dict()['county']
    else:
        return None  # Return None for missing county information
        
# Fill in missing 'Donor County' values based on ZIP codes
donations_df['Donor County'] = donations_df.apply(lambda row: get_county(row['Donor ZIP']) if pd.isnull(row['Donor County']) else row['Donor County'], axis=1)

# State

In [42]:
# Donor State dirty value
max_length_value = donations_df['Donor State'].astype(str).apply(len).idxmax()
max_length = donations_df['Donor State'].astype(str).apply(len).max()
value_with_max_length = donations_df.at[max_length_value, 'Donor State']
value_with_max_length

unique_values = donations_df['Donor State'].unique()
print(unique_values)

['MS' 'AL' 'TX' 'AR' 'OK' 'CA' 'MD' 'LA' 'WI' 'CO' 'KY' 'WA' 'GA' 'TN'
 'RI' 'NC' 'VA' 'AB' 'NY' 'FL' 'OH' 'UT' 'MN' 'CT' 'DE' 'AZ' 'MO' 'SC' nan
 'AK' 'MA' 'SD' 'IN' 'MT' 'AE' 'NJ' 'IL' 'NH' 'MI' 'NM' 'IA' 'OR' 'HI'
 'KS' 'PA' 'DC' 'NV' 'SK' 'Lafayette' 'WY' 'QC' 'ME' 'NE' 'ID' 'VT' 'WV'
 'MB' 'ON' 'ND' 'PR' 'BC' 'Corvallis' 'NS' 'Phoenix' 'AP' 'VI' 'Nsw' 'NB'
 'MP' 'San Diego' 'GU' 'Grand Junction' 'AA' 'YT' 'NU' 'Greene' 'NT'
 'Abita Springs' 'NL' 'PE' 'Alicante' 'Corona Del Mar' 'Columbia Heights'
 'PW' 'MJ' 'Mooresboro' 'Ridgefield Park' 'Frankfurt Am Main' 'Cuyahoga'
 'Nogales' 'Canyon Lake' 'HM' 'Wimberley' 'Texas City' 'Austin' 'Blanco'
 'Edinburg' 'MH' 'Le Pouldu' 'Dallas' 'Hilton Head Island' 'Greendale'
 '716 Key Route Blvd' 'Naples' 'Fort Collins' 'Parker' 'Galway' 'Waverly'
 'Ontario' 'Satellite Beach' 'antioquia' 'Newport Beach' 'Johnson' 'FM'
 'Belews Creek' 'San Mateo' 'Midwest City' 'Houston' 'Claremore'
 'Rockwall' 'Lawrenceville' 'AS' 'Sacramento' 'Denver' 'Newport'


In [43]:
#Donor State
# Create a dictionary to map locations to state names
us_state_abbreviations = {
    'Lafayette': 'LA',
    'Corvallis': 'OR',
    'Phoenix': 'AZ',
    'Mooresboro': 'NC',
    'Ridgefield Park': 'NJ',
    'Nogales': 'AZ',
    'Canyon Lake': 'TX',
    'Wimberley': 'TX',
    'Texas City': 'TX',
    'Austin': 'TX',
    'Blanco': 'TX',
    'Edinburg': 'TX',
    'Dallas': 'TX',
    'Hilton Head Island': 'SC',
    'Greendale': 'WI',
    'Naples': 'FL',
    'Fort Collins': 'CO',
    'Parker': 'CO',
    'Waverly': 'IA',
    'Ontario': 'CA',
    'Satellite Beach': 'FL',
    'Newport Beach': 'CA',
    'Belews Creek': 'NC',
    'San Mateo': 'CA',
    'Midwest City': 'OK',
    'Houston': 'TX',
    'Claremore': 'OK',
    'Rockwall': 'TX',
    'Lawrenceville': 'GA',
    'Sacramento': 'CA',
    'Denver': 'CO',
    'Carrollton': 'TX',
    'Maple Grove': 'MN',
    'Morristown': 'NJ',
    'Delran': 'NJ',
    'Ardmore': 'PA',
    'Clayton': 'MO',
    'San Antonio': 'TX',
    'Seattle': 'WA',
    'Walker': 'LA',
    'Monmouth': 'NJ',
    'North Ridgeville': 'OH',
    'Philadelphia': 'PA',
    'Northampton': 'MA',
    'Graford': 'TX',
    'Grayson': 'KY',
    'Goochland': 'VA',
    'Greensboro': 'NC',
    'Katy': 'TX',
    'Fort Worth': 'TX',
    'West Haven': 'CT',
    'Blaine': 'MN',
    'Chicago': 'IL',
    'San Diego': 'CA',
    'Guam': 'GU',
    'Armed Forces Americas': 'AA',
    'American Samoa': 'AS',
    'Federated States of Micronesia': 'FM',
    'Heard Island and McDonald Islands': 'HM',
    'Marshall Islands': 'MH',
    'North Dakota': 'ND',  # Assuming 'Dakota' refers to North Dakota
    'South Dakota': 'SD',  # Assuming 'Dakota' refers to South Dakota
}

# Replace values in 'Donor State' column with state abbreviation from the dictionary
donations_df['Donor State'] = donations_df['Donor State'].replace(us_state_abbreviations)

In [44]:
max_state_length = donations_df['Donor State'].str.len().max()
max_length_states = donations_df[donations_df['Donor State'].str.len() == max_state_length]['Donor State'].unique()

print("Maximum number of characters in the 'state' column:", max_state_length)
print("State(s) with the maximum number of characters:", max_length_states)

Maximum number of characters in the 'state' column: 18.0
State(s) with the maximum number of characters: ['716 Key Route Blvd' '1601 Walnut Street']


In [45]:
num_nulls = donations_df['Donor State'].isnull().sum()
num_nulls

26317

In [46]:
# Initialize the SearchEngine
# Example function to impute missing state based on ZIP code
def impute_state_from_zip(row):
    if pd.isnull(row['Donor State']) and not pd.isnull(row['Donor ZIP']):
        # Use the uszipcode library to search by ZIP code
        zipcode_info = search.by_zipcode(row['Donor ZIP'])
        if zipcode_info:
            # If there's a result, return the state associated with the ZIP code
            return zipcode_info.state
    # Return the original state if it's not missing or if ZIP code info wasn't found
    return row['Donor State']

# Apply the function to impute missing 'Donor State' values
donations_df['Donor State'] = donations_df.apply(impute_state_from_zip, axis=1)

# Check the result
donations_df[['Donor ZIP', 'Donor State']].sample(5)


,Donor ZIP,Donor State
1382725,19122,PA
2993399,20910,MD
3520875,44223,OH
316168,55426,MN
2379557,77210,TX


In [51]:
states_with_more_than_2_chars = donations_df[donations_df['Donor State'].str.len() > 2]['Donor State'].unique()
print("Unique values in 'state' column with more than 2 characters:", states_with_more_than_2_chars)

Unique values in 'state' column with more than 2 characters: []


In [48]:
# Delete all char after "("
donations_df['Donor State'] = donations_df['Donor State'].str.split('(').str[0]

In [50]:
# Adjusted code to handle NaN values and strings
donations_df['Donor State'] = donations_df['Donor State'].apply(lambda x: x if isinstance(x, str) and len(x) <= 2 else np.nan)

# Donor City

In [52]:
# Donor City dirty value
max_length_value = donations_df['Donor City'].astype(str).apply(len).idxmax()
max_length = donations_df['Donor City'].astype(str).apply(len).max()
value_with_max_length = donations_df.at[max_length_value, 'Donor City']
value_with_max_length

# Using regular expressions to filter for values that contain a number
values_with_numbers = donations_df[donations_df['Donor City'].str.contains(r'\d', na=False)][['Donor ZIP','Donor City']]
# Print each value on a new line
values_with_numbers

,Donor ZIP,Donor City
24609,N0H1C0,R.R. 1 Berkeley
32423,SE42941,S-429 41 Saro
36543,CA,5375 Bancroft Ave
36545,CA,5375 Bancroft Ave
49964,92880,"Corona Ca, 92880"
...,...,...
3781015,D18T3K1,Dublin 18
3781047,14300,Praha 12
3790574,98027,246th Ave SE
3791538,99515,10380 Nigh Road


In [53]:
num_nulls = donations_df['Donor City'].isnull().sum()
num_nulls

982

In [54]:
# Initialize the uszipcode SearchEngine
search = SearchEngine()

# Define a function to impute missing city based on ZIP code
def impute_city_from_zip(row):
    if pd.isnull(row['Donor City']) and not pd.isnull(row['Donor ZIP']):
        # Use the uszipcode library to search by ZIP code
        zipcode_info = search.by_zipcode(row['Donor ZIP'])
        if zipcode_info and zipcode_info.city:
            # If there's a result and it includes city info, return the city
            return zipcode_info.city
    # Return the original city if it's not missing or if ZIP code info wasn't found
    return row['Donor City']

# Apply the function to impute missing 'Donor City' values
donations_df['Donor City'] = donations_df.apply(impute_city_from_zip, axis=1)

# Donor Member ID

In [55]:
# Donor Member ID dirty value
max_length_value = donations_df['Donor Member ID'].astype(str).apply(len).idxmax()
max_length = donations_df['Donor Member ID'].astype(str).apply(len).max()
value_with_max_length = donations_df.at[max_length_value, 'Donor Member ID']
value_with_max_length

# List all values 
long_year_values = donations_df[donations_df['Donor Member ID'].astype(str).apply(len) > 8]['Donor Member ID'].unique()

long_year_values

array(['56928695_removed', '58340249_removed', '60237990_removed',
       '59460287_removed', '75524768_removed', '90362893_removed',
       '56694874_removed', '91033605(MNM', '101010101'], dtype=object)

In [56]:
# Define a regular expression to match special characters
special_char_pattern = r'[^0-9a-zA-Z]'

# Find rows where 'Donor Member ID' contains special characters
special_char_values = donations_df[donations_df['Donor Member ID'].str.contains(special_char_pattern, na=False)]['Donor Member ID']

# Display the values that contain special characters
special_char_values

259047     56928695_removed
1176798    58340249_removed
1857867    60237990_removed
2002072    59460287_removed
2066858    75524768_removed
2551450    90362893_removed
3326300    56694874_removed
3396540        91033605(MNM
Name: Donor Member ID, dtype: object

In [57]:
# Directly update 'Donor Member ID' with the extracted numbers
donations_df['Donor Member ID'] = donations_df['Donor Member ID'].str.extract('(\d+)')

In [58]:
# Set values to NaN where the length of the 'Donor Member ID' string is greater than 8
donations_df.loc[donations_df['Donor Member ID'].str.len() > 8, 'Donor Member ID'] = np.nan

In [59]:
# Convert 'Participant Contact ID' to pandas nullable integer type
donations_df['Donor Member ID'] = pd.to_numeric(donations_df['Donor Member ID'], errors='coerce').astype('Int64')
donations_df['Donor Member ID']

0          63278064
1          80700130
2          80700130
3          75610532
4          80504435
             ...   
3807371    90557213
3807372    91018428
3807373    91015103
3807374    90967760
3807375    91053627
Name: Donor Member ID, Length: 3807376, dtype: Int64

In [60]:
num_nulls = donations_df['Donor Member ID'].isnull().sum()
num_nulls

4941

# Gift Amount($)

In [61]:
# Gift amount

# Find the row with the maximum length in the 'Gift Amount($)' column
max_length_value = donations_df['Gift Amount($)'].astype(str).apply(len).idxmax()

# Extract the 'Fiscal Year' and 'Gift Amount($)' values for that row
value_with_max_length = donations_df.loc[max_length_value, ['Gift Amount($)']]

# Print the result
value_with_max_length


Gift Amount($)    25(3/7/2016 10:21
Name: 2975192, dtype: object

In [62]:
# The pattern '[^0-9.]' matches any character that is not a digit or a period
special_char_values = donations_df[donations_df['Gift Amount($)'].str.contains('[^0-9.]', na=False)]['Gift Amount($)']

# Display the values that contain special characters
special_char_values

1065707      8/29/2014 20:31
2951655      8/28/2016 15:59
2975192    25(3/7/2016 10:21
3109122       6/22/2016 9:36
3142276      5/10/2016 14:23
3376382      7/14/2017 17:39
Name: Gift Amount($), dtype: object

In [63]:
# The pattern '[^0-9.]' matches any character that is not a digit or a period
condition = donations_df['Gift Amount($)'].str.contains('[^0-9.]', na=False)

# Replace these values with NaN
donations_df.loc[condition, 'Gift Amount($)'] = np.nan

In [64]:
# Define a custom function to convert to float
def convert_to_float(value):
    if value is not None:
        return float(value)
    return None

# Apply the custom function to 'Gift Amount($)' column
donations_df['Gift Amount($)'] = donations_df['Gift Amount($)'].apply(convert_to_float)

In [65]:
donations_df['Gift Amount($)'].head()

0    100.0
1     50.0
2     25.0
3     70.0
4     25.0
Name: Gift Amount($), dtype: float64

In [66]:
num_nulls = donations_df['Gift Amount($)'].isnull().sum()
num_nulls

6

# Ledger Transaction Amount

In [67]:
#Ledger Transaction Amount
# Convert the column values to strings
donations_df['Ledger Transaction Amount'] = donations_df['Ledger Transaction Amount'].astype(str)

# Calculate the length of each value in the column
donations_df['Length'] = donations_df['Ledger Transaction Amount'].str.len()

# Find the maximum length
max_length = donations_df['Length'].max()

# Display the maximum length
print("Maximum Number of Characters:", max_length)

# Find and display the value with the maximum length
max_length_value = donations_df[donations_df['Length'] == max_length]['Ledger Transaction Amount'].values[0]

print("Maximum Number of Characters:", max_length)
print("Value with Maximum Characters:", max_length_value)

Maximum Number of Characters: 26
Maximum Number of Characters: 26
Value with Maximum Characters: 100(TeamRaiser Participant


In [68]:
# The pattern '[^0-9.]' matches any character that is not a digit or a period
special_char_values = donations_df[donations_df['Ledger Transaction Amount'].str.contains('[^0-9.]', na=False)]['Ledger Transaction Amount']

# Display the values that contain special characters
special_char_values

46998          TeamRaiser Participant
175502                TeamRaiser Team
209938         TeamRaiser Participant
321402         TeamRaiser Participant
472944         TeamRaiser Participant
485629         TeamRaiser Participant
503662         TeamRaiser Participant
540563     100(TeamRaiser Participant
622886         TeamRaiser Participant
652437         TeamRaiser Participant
705376     100(TeamRaiser Participant
801593         TeamRaiser Participant
992076         TeamRaiser Participant
1065707        TeamRaiser Participant
1336119        TeamRaiser Participant
1551042        TeamRaiser Participant
1774627        TeamRaiser Participant
1842656        TeamRaiser Participant
1943101        TeamRaiser Participant
2036313        TeamRaiser Participant
2108177        TeamRaiser Participant
2159702        TeamRaiser Participant
2224841        TeamRaiser Participant
2323149        TeamRaiser Participant
2487511        TeamRaiser Participant
2492079        TeamRaiser Participant
2721961     

In [69]:
# Directly update 'Donor Member ID' with the extracted numbers
donations_df['Ledger Transaction Amount'] = donations_df['Ledger Transaction Amount'].str.extract('(\d+)')

In [70]:
# Assuming you want to find the row at index 540563
row_index_to_find = 705376

# Retrieve the row at index 540563
row_at_index_540563 = donations_df.iloc[row_index_to_find]['Ledger Transaction Amount']
row_at_index_540563

'100'

In [71]:
# Define a custom function to convert to float
def convert_to_float(value):
    if value is not None:
        return float(value)
    return None

# Apply the custom function to 'Ledger Transaction Amount' column
donations_df['Ledger Transaction Amount'] = donations_df['Ledger Transaction Amount'].apply(convert_to_float)

In [72]:
donations_df['Ledger Transaction Amount']

0          100.0
1           50.0
2           25.0
3           70.0
4           25.0
           ...  
3807371    100.0
3807372     50.0
3807373     35.0
3807374     50.0
3807375     35.0
Name: Ledger Transaction Amount, Length: 3807376, dtype: float64

# Participant Contact ID

In [73]:
# Convert 'Participant Contact ID' column to string type
donations_df['Participant Contact ID'] = donations_df['Participant Contact ID'].astype(str)

# Define a regular expression to match special characters
special_char_pattern = r'[^0-9a-zA-Z]'

# Find rows where 'Participant Contact ID' contains special characters
special_char_values = donations_df[donations_df['Participant Contact ID'].str.contains(special_char_pattern, na=False)]['Participant Contact ID']

# Display the values that contain special characters
special_char_values

0           7744894.0
1           7744894.0
2           7744894.0
3          10244498.0
4          10244498.0
              ...    
3807369    15614806.0
3807370    15614806.0
3807371    15614806.0
3807372    15614806.0
3807373    15614806.0
Name: Participant Contact ID, Length: 3765007, dtype: object

In [74]:
num_nulls = donations_df['Participant Contact ID'].isnull().sum()
num_nulls

0

In [75]:
# Check the cleaned data
max_char = max(len(str(id)) for id in donations_df['Participant Contact ID'])
max_char

10

In [76]:
# Filter rows where 'Participant Contact ID' has more than 8 characters
filtered_df = donations_df[donations_df['Participant Contact ID'].str.len() > 8]

# Display only the 'Participant Contact ID' column from the filtered DataFrame
filtered_id_column = filtered_df['Participant Contact ID']
filtered_id_column

0           7744894.0
1           7744894.0
2           7744894.0
3          10244498.0
4          10244498.0
              ...    
3807369    15614806.0
3807370    15614806.0
3807371    15614806.0
3807372    15614806.0
3807373    15614806.0
Name: Participant Contact ID, Length: 3765007, dtype: object

In [77]:
# Convert 'Participant Contact ID' to pandas nullable integer type
donations_df['Participant Contact ID'] = pd.to_numeric(donations_df['Participant Contact ID'], errors='coerce').astype('Int64')
donations_df['Participant Contact ID']


0           7744894
1           7744894
2           7744894
3          10244498
4          10244498
             ...   
3807371    15614806
3807372    15614806
3807373    15614806
3807374        <NA>
3807375        <NA>
Name: Participant Contact ID, Length: 3807376, dtype: Int64

# Donor City

In [78]:
num_nulls = donations_df['Donor City'].isnull().sum()
num_nulls

925

In [79]:
# Filter rows where 'Donor City' contains numbers
values_with_numbers = donations_df[donations_df['Donor City'].str.contains(r'\d', na=False)]

# Display only the 'Donor City' column from the filtered DataFrame
filtered_city_column = values_with_numbers['Donor City']
filtered_city_column

24609        R.R. 1 Berkeley
32423          S-429 41 Saro
36543      5375 Bancroft Ave
36545      5375 Bancroft Ave
49964       Corona Ca, 92880
                 ...        
3781015            Dublin 18
3781047             Praha 12
3790574         246th Ave SE
3791538      10380 Nigh Road
3801927                60201
Name: Donor City, Length: 416, dtype: object

In [80]:
# Create a boolean mask to identify city names containing numbers
contains_numbers_mask = donations_df['Donor City'].str.contains(r'\d', na=False)

# Replace city names containing numbers with NaN
donations_df.loc[contains_numbers_mask, 'Donor City'] = np.nan

In [81]:
# Initialize the SearchEngine
search = SearchEngine()

# Define a function to impute missing 'Donor City' based on 'Donor ZIP'
def impute_city(row):
    # Check if 'Donor City' is missing and 'Donor ZIP' is not missing
    if pd.isna(row['Donor City']) and not pd.isna(row['Donor ZIP']):
        # Lookup the ZIP code to get city name
        result = search.by_zipcode(row['Donor ZIP'])
        if result:
            # Return the city name if found
            return result.city
    # Return the original city if not missing or if no city found for the ZIP code
    return row['Donor City']

# Apply the function to the DataFrame
donations_df['Donor City'] = donations_df.apply(impute_city, axis=1)

In [82]:
donations_df['Donor City']

0              Jackson
1           Birmingham
2           Birmingham
3               Frisco
4               Austin
              ...     
3807371      Fitchburg
3807372         Marion
3807373        Madison
3807374    Minneapolis
3807375       Waukesha
Name: Donor City, Length: 3807376, dtype: object

# Donor ConsID

In [83]:
num_nulls = donations_df['Donor ConsID'].isnull().sum()
num_nulls

1

In [84]:
donations_df.columns

Index(['Security Category Name', 'Event ID', 'Public Event Name',
       'Fiscal Year', 'Event Date', 'Campaign Title', 'Campaign ID',
       'Gift Amount($)', 'Date Recorded', 'Gift Type', 'Gift Payment Method',
       'Offline Status', 'Soft Credit Type', 'Is Registration?',
       'Donor ConsID', 'Donor Member ID', 'Donor Affiliate Code',
       'Donor Gender', 'Donor Accept Email', 'Donor Opt-out Date',
       'Donor Opt-out Method', 'Donor Email Status', 'Donor City',
       'Donor State', 'Donor County', 'Donor ZIP', 'Donor Employer',
       'Donor Connection to MS', 'Participant Contact ID',
       'Participant Member ID', 'Participation Type Name',
       'Registration Active Status', 'Registration Date',
       'Participant Goal($)', 'Is Prior Participant', 'Is Team Captain',
       'Additional Gift Amount($)', 'Team Name', 'Team', 'Team Creation Date',
       'Original Value Transacted', 'Net Transaction Amount',
       'Ledger Transaction Amount', 'Proxy Type', 'User Confirm

In [85]:
# The pattern '[^0-9.]' matches any character that is not a digit or a period
special_char_pattern = r'[^0-9.]'

# Filter rows where 'Donor ConsID' contains special characters
special_char_rows = donations_df[donations_df['Donor ConsID'].str.contains(special_char_pattern, regex=True, na=False)]

# Display the relevant columns for the rows with special characters
relevant_columns = ['Donor ConsID', 'Donor Member ID', 'Campaign ID', 'Event ID', 'Team', 'Participant Contact ID', 'Participant Member ID', 'Batch ID']
special_char_data = special_char_rows[relevant_columns]

# Corrected line: Display the DataFrame with the values that contain special characters
special_char_data

,Donor ConsID,Donor Member ID,Campaign ID,Event ID,Team,Participant Contact ID,Participant Member ID,Batch ID
485629,1941829(42444387,<NA>,35566,20976,316590,61475942,Event Participant - Cyclist,NaN
2487511,14525053(89342577,<NA>,35582,24838,400439,44428153,"Cyclist - June 1, 2015 to day of ride",NaN


In [86]:
# Clean up 'Donor ConsID' values
donations_df['Donor ConsID'] = donations_df['Donor ConsID'].astype(str)  # Ensure 'Donor ConsID' is a string
donations_df['Donor ConsID'] = donations_df['Donor ConsID'].str.split('(').str[0]  # Remove everything after '('
donations_df['Donor ConsID'] = donations_df['Donor ConsID'].str.split('.').str[0]  # Remove decimal part

In [87]:
# List all values 
long_year_values = donations_df[donations_df['Donor ConsID'].astype(str).apply(len) > 7]['Donor ConsID'].unique()

long_year_values

array(['11527263', '10244498', '11415664', ..., '16083016', '16033523',
       '16122821'], dtype=object)

In [88]:
# Assuming you want to find the row at index 540563
row_index_to_find = 2487511

# Retrieve the row at index 540563
row_at_index_540563 = donations_df.iloc[row_index_to_find]['Donor ConsID']
row_at_index_540563

'14525053'

In [89]:
# Convert 'Donor ConsID' to pandas nullable integer type
donations_df['Donor ConsID'] = pd.to_numeric(donations_df['Donor ConsID'], errors='coerce').astype('Int64')
donations_df['Donor ConsID']

0           7828995
1          11527263
2          11527263
3          10244498
4          11415664
             ...   
3807371    15614806
3807372    16085762
3807373    16083016
3807374    16033523
3807375    16122821
Name: Donor ConsID, Length: 3807376, dtype: Int64

# Event ID

In [90]:
num_nulls = donations_df['Event ID'].isnull().sum()
num_nulls

0

In [91]:
# The pattern '[^0-9.]' matches any character that is not a digit or a period
special_char_pattern = r'[^0-9.]'

# Filter rows where 'Donor ConsID' contains special characters
special_char_rows = donations_df[donations_df['Event ID'].str.contains(special_char_pattern, regex=True, na=False)]

# Display the relevant columns for the rows with special characters
relevant_columns = ['Donor ConsID', 'Donor Member ID', 'Campaign ID', 'Event ID', 'Team', 'Participant Contact ID', 'Participant Member ID', 'Batch ID']
special_char_data = special_char_rows[relevant_columns]

# Corrected line: Display the DataFrame with the values that contain special characters
special_char_data

,Donor ConsID,Donor Member ID,Campaign ID,Event ID,Team,Participant Contact ID,Participant Member ID,Batch ID
2951655,<NA>,<NA>,20,Bike MS: City to Shore Ride 2016,452046,45404988,Cyclist,NaN
3109122,90341958,<NA>,40,Bike MS: Sam's Club Round-Up Ride 2016,<NA>,76816158,Cyclist,NaN


In [92]:
# Define the search strings
search_event_name = 'Bike MS: City to Shore Ride 2016'
#search_campaign_id = 20
search_team = 452046

# Filter rows where 'Public Event Name' contains the search string and 'Campaign ID' equals 20 and 'Team' equals 452046
matching_rows = donations_df[
    (donations_df['Public Event Name'].str.contains(search_event_name, case=False, na=False)) &
   
    (donations_df['Team'] == search_team)
]

# Select the 'Event ID' column from the matching_rows DataFrame
filtered_event_ids = matching_rows['Event ID'].unique()

# Display the unique Event IDs associated with the campaign
filtered_event_ids

array(['27061'], dtype=object)

In [93]:
# Define the replacement value
replacement_value = '27149'  # Replace with the desired value

# Conditionally update 'Event ID'
donations_df.loc[donations_df['Event ID'] == "Bike MS: Sam's Club Round-Up Ride 2016", 'Event ID'] = replacement_value

In [94]:
# Define the replacement value
replacement_value = '27061'  # Replace with the desired value

# Conditionally update 'Event ID'
donations_df.loc[donations_df['Event ID'] == "Bike MS: City to Shore Ride 2016", 'Event ID'] = replacement_value

In [95]:
# Assuming you want to find the row at index 540563
row_index_to_find = 2951655

# Retrieve the row at index 540563
row_at_index_540563 = donations_df.iloc[row_index_to_find]['Event ID']
row_at_index_540563

'27061'

In [96]:
# Convert 'Event ID' to pandas nullable integer type
donations_df['Event ID'] = pd.to_numeric(donations_df['Event ID'], errors='coerce').astype('Int64')
donations_df['Event ID']

0          17966
1          17966
2          17966
3          17966
4          17966
           ...  
3807371    28198
3807372    28198
3807373    28198
3807374    28198
3807375    28198
Name: Event ID, Length: 3807376, dtype: Int64

# Additional Gift Amount($)

In [97]:
num_nulls = donations_df['Additional Gift Amount($)'].isnull().sum()
num_nulls

5

In [98]:
# Convert 'Additional Gift Amount($)' column to float, coercing errors to NaN
donations_df['Additional Gift Amount($)'] = pd.to_numeric(donations_df['Additional Gift Amount($)'], errors='coerce')

donations_df['Additional Gift Amount($)']


0            0.0
1            0.0
2            0.0
3            0.0
4            0.0
           ...  
3807371    100.0
3807372    100.0
3807373    100.0
3807374      0.0
3807375      0.0
Name: Additional Gift Amount($), Length: 3807376, dtype: float64

# Search for values

In [212]:
# Define the specific 'Donor Member ID' you're looking for
specific_member_id = '22517'

# Filter the DataFrame for rows where 'Donor Member ID' matches the specific ID
matching_rows = donations_df[donations_df['Event ID'] == specific_member_id]
matching_rows

,Security Category Name,Event ID,Public Event Name,Fiscal Year,Event Date,Campaign Title,Campaign ID,Gift Amount($),Date Recorded,Gift Type,Gift Payment Method,Offline Status,Soft Credit Type,Is Registration?,Donor ConsID,Donor Member ID,Donor Affiliate Code,Donor Gender,Donor Accept Email,Donor Opt-out Date,Donor Opt-out Method,Donor Email Status,Donor City,Donor State,Donor County,Donor ZIP,Donor Employer,Donor Connection to MS,Participant Contact ID,Participant Member ID,Participation Type Name,Registration Active Status,Registration Date,Participant Goal($),Is Prior Participant,Is Team Captain,Additional Gift Amount($),Team Name,Team,Team Creation Date,Original Value Transacted,Net Transaction Amount,Ledger Transaction Amount,Proxy Type,User Confirmation Code,Source Code Type,Source Code Text,Sub Source Code Text,Batch ID,Length


In [94]:
donations_df.dtypes

Security Category Name                object
Event ID                               Int64
Public Event Name                     object
Fiscal Year                            Int64
Event Date                            object
Campaign Title                        object
Campaign ID                           object
Gift Amount($)                       float64
Date Recorded                 datetime64[ns]
Gift Type                             object
Gift Payment Method                   object
Offline Status                        object
Soft Credit Type                      object
Is Registration?                      object
Donor ConsID                           Int64
Donor Member ID                        Int64
Donor Affiliate Code                  object
Donor Gender                          object
Donor Accept Email                    object
Donor Opt-out Date                    object
Donor Opt-out Method                  object
Donor Email Status                    object
Donor City

# Primary Key

In [346]:
# drop column
#donations_df = donations_df.drop(columns=['participant_ID'])

# Drop unnecessary columns

In [99]:
columns_to_keep = ['Donor Gender', 'Donor County', 'Donor State', 'Donor Member ID', 
                   'Donor City', 'Additional Gift Amount($)', 'Donor ConsID', 'Gift Amount($)', 
                   'Date Recorded', 'Gift Type', 'Fiscal Year','Ledger Transaction Amount','Participant Contact ID',]

donations_df = donations_df[columns_to_keep]
donations_df.columns


Index(['Donor Gender', 'Donor County', 'Donor State', 'Donor Member ID',
       'Donor City', 'Additional Gift Amount($)', 'Donor ConsID',
       'Gift Amount($)', 'Date Recorded', 'Gift Type', 'Fiscal Year',
       'Ledger Transaction Amount', 'Participant Contact ID'],
      dtype='object')

# change name columns

In [100]:
donations_df.rename(
    columns={"Donor Gender": "d_gender", "Donor County": "d_county","Donor State":"d_state","Donor Member ID":"donationID","Donor City":"d_city","Additional Gift Amount($)":"do_add_gift_amt","Donor ConsID":"donorID","Gift Amount($)":"do_gift_amt","Date Recorded":"do_date_recorded","Gift Type":"do_gift_type","Fiscal Year":"do_fiscal_yr","Ledger Transaction Amount":"do_amt_collected",'Participant Contact ID':'p_contactID'},
    inplace=True,
)

donations_df

,d_gender,d_county,d_state,donationID,d_city,do_add_gift_amt,donorID,do_gift_amt,do_date_recorded,do_gift_type,do_fiscal_yr,do_amt_collected,p_contactID
0,Female,Hinds,MS,63278064,Jackson,0.0,7828995,100.0,2012-09-14 17:34:00,Online,2013,100.0,7744894
1,Female,Jefferson,AL,80700130,Birmingham,0.0,11527263,50.0,2012-09-21 14:10:00,Online,2013,50.0,7744894
2,Female,Jefferson,AL,80700130,Birmingham,0.0,11527263,25.0,2012-09-21 10:43:00,Online,2013,25.0,7744894
3,Male,Denton,TX,75610532,Frisco,0.0,10244498,70.0,2012-10-02 15:58:00,Online,2013,70.0,10244498
4,Female,Hays,TX,80504435,Austin,0.0,11415664,25.0,2012-08-17 14:15:00,Online,2013,25.0,10244498
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3807371,Female,Dane,WI,90557213,Fitchburg,100.0,15614806,100.0,2017-05-16 08:57:00,Online,2017,100.0,15614806
3807372,Female,Linn,IA,91018428,Marion,100.0,16085762,50.0,2017-05-18 16:49:00,Online,2017,50.0,15614806
3807373,Female,Dane,WI,91015103,Madison,100.0,16083016,35.0,2017-05-17 20:07:00,Online,2017,35.0,15614806
3807374,Unknown,Hennepin,MN,90967760,Minneapolis,0.0,16033523,50.0,2017-05-02 07:57:00,Online,2017,50.0,<NA>


# Tables

In [101]:
donations_df.columns

Index(['d_gender', 'd_county', 'd_state', 'donationID', 'd_city',
       'do_add_gift_amt', 'donorID', 'do_gift_amt', 'do_date_recorded',
       'do_gift_type', 'do_fiscal_yr', 'do_amt_collected', 'p_contactID'],
      dtype='object')

In [102]:
# donation
donation = donations_df[['donorID','p_contactID','do_gift_amt','do_date_recorded','do_gift_type','do_add_gift_amt','do_amt_collected','do_fiscal_yr']].copy()
#sector_industry.replace("Nan", np.nan, inplace=True)

donation.to_csv('C:/Users/SANTIAGO/OneDrive - Nova Scotia Community College/4 Winter Semester 2024/DBAS 3018 Data Movement and Integration/Assignment 2/Datasets/Table/donation.csv', index=False)

In [103]:
donation.dtypes

donorID                      Int64
p_contactID                  Int64
do_gift_amt                float64
do_date_recorded    datetime64[ns]
do_gift_type                object
do_add_gift_amt            float64
do_amt_collected           float64
do_fiscal_yr                 Int64
dtype: object

In [104]:
# donor
donor = donations_df[['donorID','d_gender','d_county','d_state','d_city']].copy()

# Drop rows with duplicate 'donorID', keeping the first occurrence
donor_cleaned = donor.drop_duplicates(subset='donorID', keep='first', inplace=True)

# drop rows where donorID is null
donor_cleaned = donor.loc[donations_df['donorID'].notnull()].copy()

donor_cleaned.to_csv('C:/Users/SANTIAGO/OneDrive - Nova Scotia Community College/4 Winter Semester 2024/DBAS 3018 Data Movement and Integration/Assignment 2/Datasets/Table/donor.csv', index=False)

In [105]:
donor_cleaned.shape[0]

1955809

In [217]:
num_distinct = donor_cleaned['donorID'].nunique()
num_distinct

1955809

In [100]:
num_nulls = donor_cleaned[donations_df['donorID'].isnull()]
num_nulls

C:\Users\SANTIAGO\AppData\Local\Temp\ipykernel_29044\1073159863.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  num_nulls = donor_cleaned[donations_df['donorID'].isnull()]


,donorID,d_gender,d_county,d_state,d_city


In [ ]:
# any rows from sector_industry where the sector_id, sector, and industry columns are all null, leave all that contains only rows where at least one of these columns has a non-null value
sector_industry = sector_industry.dropna(how='all')